In [1]:
import pandas as pd 
import numpy as np
from os import path
from glob import glob
import plotly.express as px

# import 

In [2]:
file_base = './../files'
glob(file_base+'/*')
Biospecimen_Normalized_View = pd.read_csv(file_base + "/Biospecimen_Normalized_View.csv")
Covid_History_Normalized_View = pd.read_csv(file_base + "/Covid_History_Normalized_View.csv")
Smoking_Alcohol_Drugs_View = pd.read_csv(file_base + "/Smoking_Alcohol_Drugs_View.csv")
Vial_Usage_View_1 = pd.read_csv(file_base + "/Vial_Usage_View_1.csv")
Covid_Vaccination_Status_Normalized_View = pd.read_csv(file_base + "/Covid_Vaccination_Status_Normalized_View.csv")
Biospecimen_Test_Result_Normalized_Reporting_View = pd.read_csv(file_base + "/Biospecimen_Test_Result_Normalized_Reporting_View.csv")
Biospecimen_Normalized_Reporting_View = pd.read_csv(file_base + "/Biospecimen_Normalized_Reporting_View.csv")
Reported_Health_Conditions_Brief_View = pd.read_csv(file_base + "/Reported_Health_Conditions_Brief_View.csv")
Covid_History_Normalized_Report_View = pd.read_csv(file_base + "/Covid_History_Normalized_Report_View.csv")
Cancer_Cohort_Reporting_View = pd.read_csv(file_base + "/Cancer_Cohort_Reporting_View.csv")
Demographics_View = pd.read_csv(file_base + "/Demographics_View.csv")
Reported_Health_Conditions_Description_View = pd.read_csv(file_base + "/Reported_Health_Conditions_Description_View.csv")
Covid_Vaccination_Status_Normalized_Reporting_View = pd.read_csv(file_base + "/Covid_Vaccination_Status_Normalized_Reporting_View.csv")
Participant_Visit_Date_Normalized_Duration_View =  pd.read_csv(file_base + "/Participant_Visit_Date_Normalized_Duration_View.csv")

4)	**Co-morbidities. How does a patients co-morbidity Influence their antibody measurements? Compare patients with one or multiple conditions vs patients without.** SQL query should include Research_Participant_ID, Diabetes, Hypertension, Cardiovascular_Disease, Chronic_Lung_Disease, Chronic_Kidney_Disease, Chronic_Liver_Disease, Immunosuppressive_Condition, Autoimmune_Disorder, Chronic_Neurological_Condition, Inflammatory_Disease, Viral_Infection, Cancer   vs same conditions filtering on ‘Participant does not have’,    Measurand_Antibody, Sample_Dilution, Derived_Result<br>

5)	**Compare within the group of patients with comorbidities and look at what comorbidity is ‘the worst’ to have antibody titer wise.** SQL query should include Research_Participant_ID, Diabetes, Hypertension, Cardiovascular_Disease, Chronic_Lung_Disease, Chronic_Kidney_Disease, Chronic_Liver_Disease, Immunosuppressive_Condition, Autoimmune_Disorder, Chronic_Neurological_Condition, Inflammatory_Disease, Viral_Infection, Cancer, Measurand_Antibody, Sample_Dilution, Derived_Result<br>

6)	**Is there a correlation between (low) antibody titer and alcohol/smoking/drug use?** SQL query should include Research_Participant_ID, Smoking_Or_Vaping_Status, Alcohol_Use, Drug_Use, Measurand_Antibody, Sample_Dilution, Derived_Result


In [3]:
set(Reported_Health_Conditions_Description_View.Hypertension_Description)

{'Essential Hypertension',
 'Hypertension',
 'Not Reported',
 'Participant Does Not Have'}

In [4]:
set(Reported_Health_Conditions_Description_View.Cardiovascular_Disease_Description)

{'Atherosclerotic heart disease of native coronary artery',
 'Atherosclerotic heart disease of native coronary artery | Heart Failure',
 'Atherosclerotic heart disease of native coronary artery | Other hyperlipidemia',
 'Coronary artery disease involving native heart without angina pectoris | Elevated lipoprotein (a) | Mixed hyperlipidemia',
 'Heart Failure',
 'Other hyperlipidemia',
 'Palpitations',
 'Participant Does Not Have',
 'Pure hypercholesterolemia, unspecified'}

In [5]:
Biospecimen_Test_Result_Normalized_Reporting_View.columns

Index(['Primary_Study_Cohort', 'Research_Participant_ID', 'Visit_Number',
       'Visit_Date_Normalized_Duration', 'Visit_Info_ID', 'Assay_ID',
       'Instrument_ID', 'Test_Operator_Initials', 'Assay_Kit_Lot_Number',
       'Test_Batch_ID', 'Biospecimen_Collection_to_Test_Duration',
       'Assay_Target_Organism', 'Assay_Target', 'Assay_Target_Sub_Region',
       'Measurand_Antibody', 'Assay_Replicate', 'Sample_Type',
       'Sample_Dilution', 'Interpretation', 'Derived_Result',
       'Derived_Result_Units', 'Raw_Result', 'Raw_Result_Units',
       'Positive_Control_Reading', 'Negative_Control_Reading',
       'Biospecimen_Test_Results_Comments'],
      dtype='object')

In [65]:
#Q4 / Q5
# from sklearn import preprocessing

# label_encoder = preprocessing.LabelEncoder()
# for i in comorbidities:
#     Reported_Health_Conditions_Description_View[i] = label_encoder.fit_transform(Reported_Health_Conditions_Description_View[i])


antibody_comorbidities = Biospecimen_Test_Result_Normalized_Reporting_View.merge(
    Reported_Health_Conditions_Description_View, 
    on=['Research_Participant_ID','Visit_Date_Normalized_Duration',], 
    how='left', indicator=True
)

antibody = [
    'Research_Participant_ID', 'Visit_Date_Normalized_Duration', 'Assay_Target', 'Measurand_Antibody',
    'Interpretation', 'Derived_Result','Derived_Result_Units','Raw_Result','Raw_Result_Units'
]

comorbidities = [
    'Diabetes_Description','Hypertension_Description', 
    'Cardiovascular_Disease_Description','Chronic_Lung_Disease_Description', 'Chronic_Kidney_Disease_Description',
    'Chronic_Liver_Disease_Description', 'Acute_Liver_Disease_Description',
    'Immunosuppressive_Condition_Description', 'Autoimmune_Disorder_Description',
    'Chronic_Neurological_Condition_Description', 'Chronic_Oxygen_Requirement_Description',
    'Inflammatory_Disease_Description', 'Viral_Infection','Bacterial_Infection', 'Cancer_Description'

]
# Doing this one by hand - this is to create a scale of severity
ac = antibody_comorbidities[antibody+comorbidities].copy()
ac['Interpretation_num'] = ac['Interpretation'].replace(
    {
        'Negative':0,
        'Weak Positive':1,
        'Moderate Positive':2,
        'Positive':3,
        'Strong Positive':4
    },
    inplace=False
)

# ac.loc[ac['Research_Participant_ID'] == '14_M45025']
# ac['z_derived_results'] = ac.groupby('Derived_Result_Units').transform(lambda x: (x - x.mean()) / x.std())['Derived_Result']
# ac['norm_derived_results'] = ac.groupby('Derived_Result_Units').transform(lambda x: (x - x.min()) / (x.max() - x.min()))['Derived_Result']

# ac['z_derived_results'] = ac.groupby('Derived_Result_Units').transform(lambda x: (x - x.mean()) / x.std())['Derived_Result']
ac['norm_raw_results'] = ac['Raw_Result'].transform(lambda x: (x - x.min()) / (x.max() - x.min()))


ac = ac.dropna(axis = 0).reset_index() # dropping rows where there is Assay information but not Comorbidities
ac.head()

,index,Research_Participant_ID,Visit_Date_Normalized_Duration,Assay_Target,Measurand_Antibody,Interpretation,Derived_Result,Derived_Result_Units,Raw_Result,Raw_Result_Units,...,Immunosuppressive_Condition_Description,Autoimmune_Disorder_Description,Chronic_Neurological_Condition_Description,Chronic_Oxygen_Requirement_Description,Inflammatory_Disease_Description,Viral_Infection,Bacterial_Infection,Cancer_Description,Interpretation_num,norm_raw_results
0,0,14_I10465,0,Spike,IgG,Negative,0.078,COI450,0.02600,OD,...,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,Crohn's disease,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,0,0.009581
1,1,14_I10465,231,Spike,IgG,Positive,6.032,COI450,2.42700,OD,...,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,Crohn's disease,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,3,0.894361
2,2,14_I10465,231,Spike,IgG,Strong Positive,1493.000,Au/mL,0.73000,OD,...,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,Crohn's disease,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,4,0.269008
3,5,14_I11895,0,Spike,IgG,Strong Positive,716.000,Au/mL,0.79200,OD,...,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,Crohn's disease,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,4,0.291856
4,6,14_I11895,0,Spike,IgG,Positive,6.134,COI450,2.04262,OD,...,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,Crohn's disease,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,3,0.752715


In [66]:
np.corrcoef(ac['Interpretation_num'],ac['norm_raw_results'])

array([[1.        , 0.14034428],
       [0.14034428, 1.        ]])

In [54]:
comorbidities_sub = []
healthy_sub = []
for i, ID in enumerate(ac['Research_Participant_ID']):
    counter = 0
    for k in comorbidities:
        if ac[k][i] not in ['Participant Does Not Have', 'Not Reported']:
#             print(ID, ac['Visit_Date_Normalized_Duration'][i],ac[k][i])
            counter +=1 
    if counter >0:
        comorbidities_sub.append(i)
    else:
        healthy_sub.append(i)
        

#subset of data where Comorbidities: 'Participant Does Not Have' or 'Not Reported'
co_subset = ac.iloc[comorbidities_sub].reset_index(drop=True)
healthy_subset = ac.iloc[healthy_sub].reset_index(drop=True)

In [55]:
'''things to look at: 

Hypertension_Description
Cardiovascular_Disease_Description
Immunosuppressive_Condition_Description
Autoimmune_Disorder_Description
Chronic_Neurological_Condition_Description
Inflammatory_Disease_Description
Cancer_Description

'''
look_at = [
'Hypertension_Description',
'Cardiovascular_Disease_Description',
'Immunosuppressive_Condition_Description',
'Autoimmune_Disorder_Description',
'Chronic_Neurological_Condition_Description',
'Inflammatory_Disease_Description',
'Cancer_Description'
]

In [56]:
len(co_subset)

545

In [57]:
number_of_co = []
types_of_co = []
for i in range(len(co_subset)):
    count = 0
    count2 = 0
    for look in look_at:
        if co_subset[look][i] not in ['Participant Does Not Have', 'Not Reported']:
            count2 += 1
            if len(co_subset[look][i].split('|')) > 1:
                count += len(co_subset[look][i].split('|'))
            else:
                count += 1
           
    number_of_co.append(count)
    types_of_co.append(count2)

co_subset['Number_of_Comorbidities'] = number_of_co
co_subset['Number_of_Comorbidity_Types'] = types_of_co 
co_subset['Number_of_Comorbidities'] = co_subset['Number_of_Comorbidities'].astype(str)
co_subset['Number_of_Comorbidity_Types'] = co_subset['Number_of_Comorbidity_Types'].astype(str)


# co_subset[['Research_Participant_ID','Interpretation','Number_of_Comorbidities']]

In [58]:
test = co_subset.copy()
test = test.loc[test['Number_of_Comorbidities'] == 1]

In [59]:
co_subset.head()

,index,Research_Participant_ID,Visit_Date_Normalized_Duration,Assay_Target,Measurand_Antibody,Interpretation,Derived_Result,Derived_Result_Units,Raw_Result,Raw_Result_Units,...,Autoimmune_Disorder_Description,Chronic_Neurological_Condition_Description,Chronic_Oxygen_Requirement_Description,Inflammatory_Disease_Description,Viral_Infection,Bacterial_Infection,Cancer_Description,norm_raw_results,Number_of_Comorbidities,Number_of_Comorbidity_Types
0,0,14_I10465,0,Spike,IgG,Negative,0.078,COI450,0.02600,OD,...,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,Crohn's disease,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,0.009581,1,1
1,1,14_I10465,231,Spike,IgG,Positive,6.032,COI450,2.42700,OD,...,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,Crohn's disease,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,0.894361,1,1
2,2,14_I10465,231,Spike,IgG,Strong Positive,1493.000,Au/mL,0.73000,OD,...,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,Crohn's disease,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,0.269008,1,1
3,5,14_I11895,0,Spike,IgG,Strong Positive,716.000,Au/mL,0.79200,OD,...,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,Crohn's disease,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,0.291856,2,2
4,6,14_I11895,0,Spike,IgG,Positive,6.134,COI450,2.04262,OD,...,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,Crohn's disease,Participant Does Not Have,Participant Does Not Have,Participant Does Not Have,0.752715,2,2


In [63]:
np.corrcoef(co_subset['Interpretation'],co_subset['Derived_Result'])

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [60]:
px.scatter(co_subset, x = 'norm_raw_results', color = 'Interpretation'
    )

In [49]:
# #Does the number of comorbidities increase interpretation
# import plotly
# # co_subset[['Research_Participant_ID','Interpretation','Number_of_Comorbidities']]

# # x axis values
# x = test['Interpretation']
# # corresponding y axis values
# y = test['Number_of_Comorbidities']
  
# import plotly.express as px
# fig = px.histogram(co_subset, x="Number_of_Comorbidities", y="Interpretation", color="Number_of_Comorbidities",
#                    marginal="box", # or violin, rug
#                    hover_data=co_subset.columns)
# fig.show()


In [ ]:
Reported_Health_Conditions_Description_View.loc[
    Reported_Health_Conditions_Description_View['Viral_Infection'] != 'Participant Does Not Have'
]

In [ ]:
Biospecimen_Test_Result_Normalized_Reporting_View[
    Biospecimen_Test_Result_Normalized_Reporting_View['Research_Participant_ID'] == '14_T53821']

In [ ]:
Reported_Health_Conditions_Description_View.columns

In [ ]:
Reported_Health_Conditions_Description_View.columns

In [69]:
#Q6 (ASSUMPTION - )
Smoking_Alcohol_Drugs_View.head()
Biospecimen_Normalized_Reporting_View.head()

bio_drugs = Biospecimen_Test_Result_Normalized_Reporting_View.merge(
    Smoking_Alcohol_Drugs_View, on=['Research_Participant_ID','Visit_Number'], how='left', indicator=True
)
bio_drugs.columns
bg = bio_drugs[['Research_Participant_ID','Assay_Target', 'Measurand_Antibody',
                'Interpretation', 'Derived_Result','Derived_Result_Units',
                'Smoking_Or_Vaping_Status','Alcohol_Use', 'Drug_Use', 'Drug_Type',]].copy()

# changing Interpretation (cat) to numeric, since i cannot compare Derived_Result due to different units
# bg['Interpretation'].replace(
#     {
#         'Negative':0,
#         'Weak Positive':1,
#         'Moderate Positive':2,
#         'Positive':3,
#         'Strong Positive':4
#     },
#     inplace=True
# )


## Never mind, there are no reported drug uses that have serology assays?????

In [75]:
set(bg['Smoking_Or_Vaping_Status'])
set(bg['Alcohol_Use'])
set(bg['Drug_Use'])
set(bg['Drug_Type'])


{nan}

In [ ]:
Biospecimen_Test_Result_Normalized_Reporting_View.loc[Biospecimen_Test_Result_Normalized_Reporting_View['Research_Participant_ID'] == '14_P83472']

In [ ]:
set(Smoking_Alcohol_Drugs_View['Drug_Use'])

In [ ]:
Smoking_Alcohol_Drugs_View.loc[Smoking_Alcohol_Drugs_View['Drug_Use'] == 'Current user, frequency unknown']

In [ ]:
Smoking_Alcohol_Drugs_View['Drug_Use'].value_counts()